In [1]:
import numpy as np
import os
import csv
import json
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import warnings
import dash
from dash import Dash, dcc, html, Input, Output, callback, dash_table, State, MATCH, ALL, Patch, State, ctx, register_page, page_container, no_update
import dash_bootstrap_components as dbc
import time
from plotly.subplots import make_subplots
from scipy.optimize import minimize
import pymysql
import dash_daq as daq

In [2]:
app = dash. Dash(__name__,  title='Робоадвайзинг - профиль', suppress_callback_exceptions=True)

In [3]:
class DataStorage:
   def __init__(self):
       self.df = pd.DataFrame()

data_storage = DataStorage()
data_storage_for_table = DataStorage()

'''Цвета для текста'''
colors =   {
            'background': '#F28A30',
            'text': '#F07B3F'
            }
"""Переменые с опциями для кнопок """
all_options = {
    'Имею гражданство только РФ': ['Для семьи', 'Для предпренимателя', 'Не важно'],
    'Имею гражданство/ВНЖ другой страны': ['Для семьи', 'Для предпренимателя', 'Не важно']
}

all_options_suit =  {
                        'Умеренный портфель': [],
                        'Консервативный портфель': [],
                        'Агресивный портфель': []
                    }

all_options_insurance = {
    'Нужно страхование': ['Медицина и страхование', 'Медицина'],
    'Не нужно страхование': []
}

all_options_passport  = {
    'Хочу другое гражданство/ВНЖ': ['Только для себя', 'Для всей семьи'],
    'Не хочу другое гражданство/ВНЖ': []
}
params = [
            'Название цели', 'Стоимость цели, в $', 'Возраст достижения цели'
         ]


########################################################################
# Контейнер для первого и второго блоков в ряд
ROW_CONTAINER = {
    'display': 'flex',  # Используем Flexbox для выравнивания блоков по горизонтали
    'justify-content': 'space-between',  # Размещение блоков с равными промежутками
    'width': '90%',  # Ширина контейнера
    'margin-left': '5%',  # Отступ слева для контейнера
    'margin-top': '5%',  # Отступ сверху для контейнера
    'height': '90%',
}
FIRST_BLOCK = {
    'width': '40%',  # Ширина блока 40%
    "margin-left": "10%",
    'height': '60%',  # Примерная высота блока
    'background-color': 'white',
    "border-top-left-radius": "20px",
    'padding': '10px 20px',  # Внутренние отступы
}

# Стили для второго блока
SECOND_BLOCK = {
    'width': '40%',  # Ширина блока 40%
    "margin-right": "10%",
    'height': '60%',  # Примерная высота блока
    'background-color': 'white',
    "border-top-right-radius": "20px",
}
########################################################################

# Отступы по краям
padding_horizontal = "2rem"

BOTTOM_STYLE = {
    "position": "fixed", 
    "height": "37%",
    "width": '72%', 
    'margin-left': '14%', 
    'margin-top': '8%', 
    'margin-bottom': '10%', 
    "top": "48%",  # Смещение вниз от верхней части двух других блоков
    "bottom": 0,  
    "padding": f"1rem {padding_horizontal}",  
    "background-color": "#FFFEFE",
    "border-bottom-left-radius": "20px",
    "border-bottom-right-radius": "20px",
    "box-sizing": "border-box",  
    "z-index": "10"  
}





In [4]:
@app.callback(
                Output('output-div', 'children'),
                Input('SAVE_DATA', 'n_clicks'),
                Input('input_name', 'value'),
                Input('input_secondname', 'value'),
                Input('input_lastname', 'value'),
                Input('input_age', 'value'),
                Input('client_year', 'value'),
                Input('client_year11', 'value'),
                Input('radioitems_type_portfel', 'value'),
                Input('strah_year', 'value'),
                Input('table_editing_simple_5', 'data')
             )
def save_data(n_click, name, second_name, last_name, age, invest_year, sum_invest, portfeltype, sum_strah, data):
    if n_click > 0:
        data_storage.df = pd.DataFrame(
                                        {
                                            'name': [name],
                                            'secondname': [second_name],
                                            'lastname': [last_name],
                                            'age': [age],
                                            'investyear': [invest_year],
                                            'suminvest': [sum_invest],
                                            'portfeltype': [portfeltype],
                                            'sumstrah': [sum_strah]
                                        }
                                        )
        data_storage_for_table.df =  pd.DataFrame( data)
        save_data_desctop(data_storage.df)
        save_data_table_desctop(data_storage_for_table.df)
        

In [5]:
def save_data_desctop(data):
    data.to_csv('/Users/erickgaydin/Desktop/USER_DATA.csv')

In [6]:
def save_data_table_desctop(data):
    data.to_csv('/Users/erickgaydin/Desktop/USER_DATA_TABLE.csv')

In [7]:
app.layout = html.Div([


    html.Div([
                        html.Div([  html.H3(children = 'Данные клиенты',
                                            style = {
                                                     'textAlign': 'center',  'color': '#B5B7B9', 'margin-bottom': '5px'
                                                    }
                                            ),
                                   dcc.Input(id="input_name", type="text", placeholder="   Имя", style={'marginRight':'10px',
                                                     'width': '45%',
                                                     'height': '5%',
                                                     'margin': '0 auto',
                                                     'margin-bottom': '5px',
                                                     'display': 'flex',
                                                     'justify-content': 'center',
                                                     'borderRadius': '20px',
                                                     'border': '2px solid #C1A068'
                                                     }),
                                    dcc.Input(id="input_secondname", type="text", placeholder="   Фамилия", style={'marginRight':'10px',
                                                     'width': '45%',
                                                     'height': '5%',
                                                     'margin': '0 auto',
                                                     'margin-bottom': '5px',
                                                     'display': 'flex',
                                                     'justify-content': 'center',
                                                     'borderRadius': '20px',
                                                     'border': '2px solid #C1A068'
                                                     }),
                                  dcc.Input(id="input_lastname", type="text", placeholder="   Отчество", style={'marginRight':'10px',
                                                     'width': '45%',
                                                     'height': '5%',
                                                     'margin': '0 auto',
                                                     'margin-bottom': '5px',
                                                     'display': 'flex',
                                                     'justify-content': 'center',
                                                     'borderRadius': '20px',
                                                     'border': '2px solid #C1A068'
                                                     }),
                                   dcc.Input(id="input_age", type="number", placeholder="   20", style={'marginRight':'10px',
                                                     'width': '45%',
                                                     'height': '5%',
                                                     'margin': '0 auto',
                                                     'margin-bottom': '5px',
                                                     'display': 'flex',
                                                     'justify-content': 'center',
                                                     'borderRadius': '20px',
                                                     'border': '2px solid #C1A068'
                                                     }),
                                  html.H3(
                                            children=[
                                                html.Span(
                                                    children=[
                                                        'Срок накопления, лет',
                                                        html.Img(
                                                            src='https://static.thenounproject.com/png/282750-200.png',  # Замените на URL вашей картинк
                                                            id = "vopros-3",
                                                            style={
                                                                "width": 15, "height": 15,
                                                                'marginLeft': '10px',  # Отступ слева от текста
                                                                'verticalAlign': 'middle'  # Выравнивание изображения по вертикали по центру
                                                            }
                                                        )
                                                    ],
                                                    style={
                                                        'display': 'inline-flex',  # Текст и картинка будут находиться на одной линии
                                                        'alignItems': 'center'  # Выравнивание по вертикали по центру
                                                    }
                                                )
                                            ],
                                            style={
                                                'textAlign': 'center',  # Центрирование текста и картинки в заголовке
                                                'color': '#B5B7B9'
                                            }
                                        ),
                                  dcc.Input(id="client_year",
                                  type="number",
                                  min=1, max=100,
                                  step=1,
                                  placeholder="  100",
                                  style={'marginRight':'10px',
                                                     'width': '45%',
                                                     'height': '5%',
                                                     'margin': '0 auto',
                                                     'display': 'flex',
                                                     'justify-content': 'center',
                                                     'borderRadius': '20px',
                                                     'border': '2px solid #C1A068'
                                                     }
                                  ),
                                  dbc.Tooltip(
                                                "Сколько полных лет Вы планируете инвестировать",
                                                target="vopros-3",  # Указываем, к какому элементу относится подсказка
                                                placement="right",  # Подсказка будет появляться слева
                                                style={
                                                    'border': '2px solid gold',  # Золотая рамка
                                                    'backgroundColor': 'white',  # Цвет фона подсказки
                                                    'color': 'black',  # Цвет текста
                                                    'padding': '5px',  # Внутренние отступы
                                                    'borderRadius': '5px',  # Скруглённые углы
                                                    'fontSize': '0.75rem'  # Уменьшаем размер текста (4 раза меньше стандартного)
                                                }
                                            ),
                                  html.H3(
                                            children=[
                                                html.Span(
                                                    children=[
                                                        'Сумма инвестиций (ежеразовая), $',
                                                        html.Img(
                                                            src='https://static.thenounproject.com/png/282750-200.png',  # Замените на URL вашей картинк
                                                            id = "vopros-2",
                                                            style={
                                                                "width": 15, "height": 15,
                                                                'marginLeft': '10px',  # Отступ слева от текста
                                                                'verticalAlign': 'middle'  # Выравнивание изображения по вертикали по центру
                                                            }
                                                        )
                                                    ],
                                                    style={
                                                        'display': 'inline-flex',  # Текст и картинка будут находиться на одной линии
                                                        'alignItems': 'center'  # Выравнивание по вертикали по центру
                                                    }
                                                )
                                            ],
                                            style={
                                                'textAlign': 'center',  # Центрирование текста и картинки в заголовке
                                                'color': '#B5B7B9'
                                            }
                                        ),
                                  dcc.Input(  id="client_year11",
                                              type="number",
                                              min=10000, max=10000000000,
                                              step=1,
                                              placeholder="   10000",
                                              style={'marginRight':'10px',
                                                                 'width': '45%',
                                                                 'height': '5%',
                                                                 'margin': '0 auto',
                                                                 'display': 'flex',
                                                                 'justify-content': 'center',
                                                                 'borderRadius': '20px',
                                                                 'border': '2px solid #C1A068'
                                                    }
                                  ),
                                  dbc.Tooltip(
                                                "Сколько денежных средств Вы готовы сразу проинвестировать",
                                                target="vopros-2",  # Указываем, к какому элементу относится подсказка
                                                placement="right",  # Подсказка будет появляться слева
                                                style={
                                                    'border': '2px solid gold',  # Золотая рамка
                                                    'backgroundColor': 'white',  # Цвет фона подсказки
                                                    'color': 'black',  # Цвет текста
                                                    'padding': '5px',  # Внутренние отступы
                                                    'borderRadius': '5px',  # Скруглённые углы
                                                    'fontSize': '0.75rem'  # Уменьшаем размер текста (4 раза меньше стандартного)
                                                }
                                            ),
                                  html.H3(
                                            children=[
                                                html.Span(
                                                    children=[
                                                        'Сумма инвестиций (ежемесячная), $',
                                                        html.Img(
                                                            src='https://static.thenounproject.com/png/282750-200.png',  # Замените на URL вашей картинк
                                                            id = "vopros-20",
                                                            style={
                                                                "width": 15, "height": 15,
                                                                'marginLeft': '10px',  # Отступ слева от текста
                                                                'verticalAlign': 'middle'  # Выравнивание изображения по вертикали по центру
                                                            }
                                                        )
                                                    ],
                                                    style={
                                                        'display': 'inline-flex',  # Текст и картинка будут находиться на одной линии
                                                        'alignItems': 'center'  # Выравнивание по вертикали по центру
                                                    }
                                                )
                                            ],
                                            style={
                                                'textAlign': 'center',  # Центрирование текста и картинки в заголовке
                                                'color': '#B5B7B9'
                                            }
                                        ),
                                  dcc.Input(  id="client_year112",
                                              type="number",
                                              min=100, max=10000000000,
                                              step=1,
                                              placeholder="   100",
                                              style={'marginRight':'10px',
                                                                 'width': '45%',
                                                                 'height': '5%',
                                                                 'margin': '0 auto',
                                                                 'display': 'flex',
                                                                 'justify-content': 'center',
                                                                 'borderRadius': '20px',
                                                                 'border': '2px solid #C1A068'
                                                    }
                                  ),
                                  dbc.Tooltip(
                                                "Сколько денежных средств Вы готовы ежемесячно инвестировать",
                                                target="vopros-20",  # Указываем, к какому элементу относится подсказка
                                                placement="right",  # Подсказка будет появляться слева
                                                style={
                                                    'border': '2px solid gold',  # Золотая рамка
                                                    'backgroundColor': 'white',  # Цвет фона подсказки
                                                    'color': 'black',  # Цвет текста
                                                    'padding': '5px',  # Внутренние отступы
                                                    'borderRadius': '5px',  # Скруглённые углы
                                                    'fontSize': '0.75rem'  # Уменьшаем размер текста (4 раза меньше стандартного)
                                                }
                                            ),
                                 ], style=FIRST_BLOCK ),
                    html.Div([  
                                html.Div([ 
                                            html.H3(
                                                children=[
                                                    'Тип портфеля',
                                                    html.A(
                                                        href='http://localhost:1317',  # Ваша ссылка
                                                        children=html.Img(
                                                            src='https://avatars.mds.yandex.net/i?id=65f7fbca99f376b3d7faa3b572d4ddec_sr-7710221-images-thumbs&n=13',  # Ссылка на изображение
                                                            id = "vopros-1",
                                                            style={
                                                                 "width": 25, "height": 25,
                                                                'marginLeft': '10px',  # Отступ слева от текста
                                                                'verticalAlign': 'middle'  # Выравнивание изображения по вертикали по центру
                                                            }
                                                        ),
                                                        target='_blank'  # Открыть ссылку в новом окне
                                                    )
                                                ],
                                                style={
                                                    'textAlign': 'center',
                                                    'color': '#B5B7B9',
                                                    'display': 'inline-flex',  # Чтобы текст и картинка отображались на одной строке
                                                    'alignItems': 'center',  # Выравнивание по вертикали
                                                     "margin-left": "30%",  # Отступ слева
                                                }
                                            ),
                                        dbc.Tooltip(
                                                "Нажмите, чтобы открыть анкету риск-профиля",
                                                target="vopros-1",  # Указываем, к какому элементу относится подсказка
                                                placement="right",  # Подсказка будет появляться слева
                                                style={
                                                    'border': '2px solid gold',  # Золотая рамка
                                                    'backgroundColor': 'white',  # Цвет фона подсказки
                                                    'color': 'black',  # Цвет текста
                                                    'padding': '5px',  # Внутренние отступы
                                                    'borderRadius': '5px',  # Скруглённые углы
                                                    'fontSize': '0.75rem'  # Уменьшаем размер текста (4 раза меньше стандартного)
                                                }
                                            ),
                                        ], style={
                                                        'justifyContent': 'center',  # Центрирование содержимого в контейнере
                                                         "margin-left": "5%",  # Отступ слева
                                                    } ),
                                dbc.RadioItems(
                                                    options=[
                                                        {"label": "Умеренный", "value": 1},
                                                        {"label": "Консервативный", "value": 2},
                                                        {"label": "Агрессивный", "value": 3,},
                                                    ],
                                                    inputStyle={"marginRight": "15px", 'borderColor' : '#C1A068'},
                                                    value=1,
                                                    id="radioitems_type_portfel",
                                                    style={  'marginRight':'10px',
                                                             'width': 200,
                                                             'height': 50,
                                                             'margin': '0 auto',
                                                             'justify-content': 'center', 
                                                             'color': '#B5B7B9'
                                                             }
                                                ),
                                html.H3(children = 'Услуги миграционного консалтинга',
                                            style = {
                                                     'textAlign': 'center',  'color': '#B5B7B9'
                                                    }
                                            ),
                                 daq.BooleanSwitch(
                                                      on=False,
                                                      label = "Хочу другое гражданство / ВНЖ",
                                                      labelPosition="top",
                                                      color="#C1A068",
                                                      persistence_type = 'memory', 
                                                      style = {'textAlign': 'center',  'color': '#B5B7B9'}
                                                    ),
                                 daq.BooleanSwitch(
                                                      on=False,
                                                      label = 'Имею гражданство только РФ', 
                                                      labelPosition="top",
                                                      color="#C1A068",
                                                      persistence_type = 'memory', 
                                                      style = {'textAlign': 'center',  'color': '#B5B7B9'}
                                                    ),
                                  html.H3(children = 'Услуги по страхованию',
                                            style = {
                                                     'textAlign': 'center',  'color': '#B5B7B9'
                                                    }
                                            ),
                                  daq.BooleanSwitch(
                                                      on=False,
                                                      color="#C1A068",
                                                      persistence_type = 'memory', 
                                                    ),
                
                                  html.H3(
                                            children=[
                                                html.Span(
                                                    children=[
                                                        'Сумма страхования в год, $',
                                                        html.Img(
                                                            src='https://static.thenounproject.com/png/282750-200.png',  # Замените на URL вашей картинк
                                                            id = "vopros-4",
                                                            style={
                                                                "width": 15, "height": 15,
                                                                'marginLeft': '10px',  # Отступ слева от текста
                                                                'verticalAlign': 'middle'  # Выравнивание изображения по вертикали по центру
                                                            }
                                                        )
                                                    ],
                                                    style={
                                                        'display': 'inline-flex',  # Текст и картинка будут находиться на одной линии
                                                        'alignItems': 'center'  # Выравнивание по вертикали по центру
                                                    }
                                                )
                                            ],
                                            style={
                                                'textAlign': 'center',  # Центрирование текста и картинки в заголовке
                                                'color': '#B5B7B9'
                                            }
                                        ),
                                  dcc.Input(  id="strah_year",
                                              type="number",
                                              min=100, max=10000000000,
                                              step=1,
                                              placeholder="   100",
                                              style={'marginRight':'10px',
                                                                 'width': '45%',
                                                                 'height': '5%',
                                                                 'margin': '0 auto',
                                                                 'display': 'flex',
                                                                 'justify-content': 'center',
                                                                 'borderRadius': '20px',
                                                                 'border': '2px solid #C1A068'
                                                                 }
                                              ),
                                dbc.Tooltip(
                                                "Сколько Вы готовы тратить в год",
                                                target="vopros-4",  # Указываем, к какому элементу относится подсказка
                                                placement="right",  # Подсказка будет появляться слева
                                                style={
                                                    'border': '2px solid gold',  # Золотая рамка
                                                    'backgroundColor': 'white',  # Цвет фона подсказки
                                                    'color': 'black',  # Цвет текста
                                                    'padding': '5px',  # Внутренние отступы
                                                    'borderRadius': '5px',  # Скруглённые углы
                                                    'fontSize': '0.75rem'  # Уменьшаем размер текста (4 раза меньше стандартного)
                                                }
                                            ),
                            
                                ], style=SECOND_BLOCK),
    ], style = ROW_CONTAINER),
                    #####Третий нижний блок с таблицей и кнопкой
                    html.Div([

                            html.H3(children = 'Финансовые цели',
                                            style = {
                                                     'textAlign': 'center',  'color': '#B5B7B9'
                                                    }
                                            ),
                            #### Редактируемая таблица для финансовых целей
                            dash_table.DataTable(
                                                    id='table_editing_simple_5',
                                                    style_data={
                                                        'whiteSpace': 'normal',
                                                        'height': 'auto',
                                                        'color': 'black',
                                                        'backgroundColor': 'white',
                                                        'border': 'none'  # Убираем все границы для данных
                                                    },
                                                    style_table={'overflowX': 'auto'},
                                                    style_header={
                                                        'backgroundColor': '#C1A068',
                                                        'color': 'white',
                                                        'fontWeight': 'bold',
                                                        'borderBottom': '1px solid black'  # Линия только между заголовками колонок и данными
                                                    },
                                                    columns=[
                                                        {'id': 'Model', 'name': 'Цель'}] + [{'id': p, 'name': p} for p in params]
                                                    ,
                                                    data=[
                                                        dict(Model=i, **{param: 0 for param in params})
                                                        for i in range(1, 6)
                                                    ],
                                                    editable=True,
                                                ),
                            html.Br(),
                            ### Кнопка РССЧИТАТЬ с вызовом на другую страницу
                            html.Div([
                                                dbc.Button(
                                                                'РАССЧИТАТЬ',
                                                                id='submit_val',
                                                                href = "http://localhost:1112/",
                                                                n_clicks=0,
                                                                style={
                                                                    'text-align': 'center', # Центрируем содержимое кнопки
                                                                    'background-color': '#0E2432',  # Темно-синий цвет заливки
                                                                    'color': 'white',  # Белый цвет текста
                                                                    'border': '2px solid #0E2432',  # Круглый контур с тем же цветом, что и фон
                                                                    'border-radius': '20px',  # Радиус скругления для круглой формы
                                                                    'padding': '10px 20px',  # Отступы внутри кнопки
                                                                    'cursor': 'pointer',  # Изменение курсора при наведении
                                                                    'font-size': '14px',  # Размер шрифта текста на кнопке
                                                                }
                                                            ),
                                                dbc.Button(
                                                                'СОХРАНИТЬ ДАННЫЕ',
                                                                id='SAVE_DATA',
                                                                n_clicks=0,
                                                                style={
                                                                    'text-align': 'center',  # Центрируем содержимое кнопки
                                                                    'background-color': '#0E2432',  # Темно-синий цвет заливки
                                                                    'color': 'white',  # Белый цвет текста
                                                                    'border': '2px solid #0E2432',  # Круглый контур с тем же цветом, что и фон
                                                                    'border-radius': '20px',  # Радиус скругления для круглой формы
                                                                    'padding': '10px 20px',  # Отступы внутри кнопки
                                                                    'cursor': 'pointer',  # Изменение курсора при наведении
                                                                    'font-size': '14px',  # Размер шрифта текста на кнопке
                                                                }
                                                            ),
                                                html.Div(id='output-div')
                                    ],
                                    style={
                                            'text-align': 'center'  # Центрирование содержимого div
                                          }
                                    )
                            ], style= BOTTOM_STYLE),
        ####ЗАкрытие основного Div
        ],      
        ### Картинка на фон страницы
        style={'background-image': 'url(https://kartinki.pics/uploads/posts/2022-12/1671725866_kartinkin-net-p-neboskrebi-kartinki-krasivo-30.jpg)',
               'background-size': '100%',
                               'left': 0,
                               "top": 0, 
               'width': '100vw',
               'height': '100vh',
               "position": "fixed", 
              }
        )



if __name__ == '__main__':
    app.run_server(port=1110, host="0.0.0.0")

